In [30]:
# Importar librerías necesarias
import os
import praw
import pandas as pd
import re
import csv

In [31]:
# Cargar credenciales
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDDIT_USERNAME = os.getenv("REDDIT_USERNAME")
REDDIT_PASSWORD = os.getenv("REDDIT_PASSWORD")

SUBREDDITS = ["Paraguay"]
USER_AGENT = "Reddit Paraguay"

# Guardar posts de r/Paraguay relacionados a criptomonedas

In [32]:
# Crear una instancia autenticada del cliente de Reddit
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
    check_for_async=False
)

In [33]:
# Definir palabras clave relacionadas a criptomonedas
CRYPTO_KEYWORDS = [
    "cripto", "criptomonedas", "bitcoin", "ethereum", "eth",
    "btc", "blockchain", "web3", "nft", "defi", "crypto", "minería", "minar",
    "p2p"
]

In [34]:
def search_crypto_posts(reddit, subreddit_name, keywords):
    resultados = []
    subreddit = reddit.subreddit(subreddit_name)

    for keyword in keywords:
        query = f'title:{keyword} OR selftext:{keyword}'
        for post in subreddit.search(query, sort="new", time_filter="all", limit=None):
            title = post.title.strip()
            body = post.selftext.strip() if post.selftext else "(sin texto)"

            # Limpiar texto para quitar links (opcional)
            body = re.sub(r'http\S+', '', body)
            title = re.sub(r'http\S+', '', title)

            resultados.append({
                "titulo": title,
                "cuerpo": body
            })
    
    return resultados

# Ejecutar búsqueda
posts = search_crypto_posts(reddit, "Paraguay", CRYPTO_KEYWORDS)

# Guardar resultados en CSV
csv_file = "data/posts_cripto_paraguay.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["titulo", "cuerpo"])
    writer.writeheader()
    for post in posts:
        writer.writerow(post)

print(f"Se guardaron {len(posts)} posts en '{csv_file}'")

Se guardaron 187 posts en 'data/posts_cripto_paraguay.csv'


# Explorar los datos almacenados

In [35]:
df = pd.read_csv(csv_file)
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  140 non-null    object
 1   cuerpo  140 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


In [36]:
# Guardar el archivo sin duplicados
csv_file = "data/posts_cripto_paraguay.csv"
df.to_csv(csv_file)

In [37]:
pd.set_option("display.max_colwidth", None)

In [38]:
df.head(5)

,titulo,cuerpo
0,Trading en Tiktok,"Que onda derrepente cada vairo que ofrece su curso, antes co el novio de la onichan nms era pesado pero ahora se reproducieron Hasta ese man que hacía tiktok de macanadas en el mercado 4 co hace TRADING, después esta ese Aquino que con un cripto cada tanto gua’u efectivizar y para colmo tiene gente boba que comenta toda la hora brooo líder espectacular quiero entrar al curso🙏🙏 tenes que irte de viaje también a dubai vos líder 🙏🙏🙏, se están volviendo peor que los colorados"
1,Percibir salario desde trabajo remoto,"Hola, el próximo mes empezaré a trabajar remoto para una multinacional extranjera. Puedo cobrar en USD o en cripto... Ustedes que todo lo saben... ¿Cuál es la mejor opción?\nSi es en USD, ¿qué bancos recomiendan y por qué? \nSi es en cripto, ¿qué billeteras recomiendan y por qué?\n ¿Sería buena idea cobrar una parte en USD y el resto en cripto? Compartan su experiencia /tips"
2,Apple gift cards,Tengo 400$ en Apple gift cards y quiero vender. Ya sea transferencia o cripto. \nVendo 100$ de gift cards por 95$
3,Recibir pagos del exterior sin tener problemas con Set,"Como paraguayo cual es la manera más simple de cobrar por trabajos que sea compatible freelancer y fiverr? Sin tener que hacer jugadas de convertir la plata a cripto, luego a euros y luego a guaraníes?"
4,Tarjetas de credito cobrando en cripto,"Alguien sabe si hay alguna forma de que ciertos bancos me habiliten tarjetas de credito si cobro en cripto? Trabajo para una empresa de afuera y cobro en dolares, para ahorrarme gastos cobro en cripto pero ulltimamente estoy queriendo quitar tarjetas de credito y no se como poder blanquear eso, consejos?"


# Criterios de etiqueta de datos [POSITIVO | NEGATIVO | NEUTRAL]

In [39]:
criterios = {
    "Etiqueta": ["Positivo", "Negativo", "Neutral"],
    "Significado": [
        "Entusiasmo, interés o experiencias positivas sobre cripto",
        "Frustración, desconfianza, crítica o experiencias negativas",
        "No expresa emoción clara; es solo una consulta, hecho o mención superficial. Habla de otro tema y menciona cripto tangencialmente o no es una opinión"
    ]
}

df_criterios = pd.DataFrame(criterios)
df_criterios

,Etiqueta,Significado
0,Positivo,"Entusiasmo, interés o experiencias positivas sobre cripto"
1,Negativo,"Frustración, desconfianza, crítica o experiencias negativas"
2,Neutral,"No expresa emoción clara; es solo una consulta, hecho o mención superficial. Habla de otro tema y menciona cripto tangencialmente o no es una opinión"


<hr/>

In [40]:
csv_file = "data/posts_cripto_paraguay_etiquetado.csv"
df_posts = pd.read_csv(csv_file, encoding="latin1", sep=";")
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       110 non-null    object
 1   cuerpo       110 non-null    object
 2   sentimiento  109 non-null    object
dtypes: object(3)
memory usage: 2.7+ KB


In [41]:
df_posts.head(1)

,titulo,cuerpo,sentimiento
0,Trading en Tiktok,"Que onda derrepente cada vairo que ofrece su curso, antes co el novio de la onichan nms era pesado pero ahora se reproducieron Hasta ese man que hac¡a tiktok de macanadas en el mercado 4 co hace TRADING, despus esta ese Aquino que con un cripto cada tanto gua?u efectivizar y para colmo tiene gente boba que comenta toda la hora brooo l¡der espectacular quiero entrar al curso???? tenes que irte de viaje tambin a dubai vos l¡der ??????, se est n volviendo peor que los colorados",Negativo


In [42]:
# Eliminar duplicados
df_posts = df_posts.drop_duplicates()
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, 0 to 108
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       95 non-null     object
 1   cuerpo       95 non-null     object
 2   sentimiento  94 non-null     object
dtypes: object(3)
memory usage: 3.0+ KB


In [43]:
df_posts[df_posts["sentimiento"].isnull()]

,titulo,cuerpo,sentimiento
27,"PayPal, crypto u otros","Buenas. Tengo el siguiente problema. Empec a trabajar online, en una plataforma que se llama Cambly. El tema es que solo pagan por PayPal. Tengo cuenta, pero entiendo que no podr retirar la plata ac en Paraguay. Estoy viendo maneras para poder efectivizar lo que gano. Las dos opciones que tengo creo que son comprar cripto monedas con lo que me pagan a paypal, o abrir una empresa en Estonia, y de ahi poder hacerme transferencias bancarias.\n\n&#x200B;\n\nHay alguna otra opcion? Estuve investigando tambien TransferWise, Neteller y Payoneer",NaN
